In [1]:
import torch.nn as nn
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
from torch.utils.data import DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import sys
import numpy as np
from PIL import Image


class ProgressBar:

    def __init__(self, width=30):
        self.width = width

    def update(self, max_value, current_value, info):
        progress = int(round(self.width * current_value / max_value))
        bar = '=' * progress + '.' * (self.width - progress)
        prefix = '{}/{}'.format(current_value, max_value)

        prefix_max_len = len('{}/{}'.format(max_value, max_value))
        buffer = ' ' * (prefix_max_len - len(prefix))

        sys.stdout.write('\r {} {} [{}] - {}'.format(prefix, buffer, bar, info))
        sys.stdout.flush()

    def new_line(self):
        print()


class Cutout(object):
    def __init__(self, num_cutouts, size, p=0.5):
        self.num_cutouts = num_cutouts
        self.size = size
        self.p = p

    def __call__(self, img):

        height, width = img.size

        cutouts = np.ones((height, width))

        if np.random.uniform() < 1 - self.p:
            return img

        for i in range(self.num_cutouts):
            y_center = np.random.randint(0, height)
            x_center = np.random.randint(0, width)

            y1 = np.clip(y_center - self.size // 2, 0, height)
            y2 = np.clip(y_center + self.size // 2, 0, height)
            x1 = np.clip(x_center - self.size // 2, 0, width)
            x2 = np.clip(x_center + self.size // 2, 0, width)

            cutouts[y1:y2, x1:x2] = 0

        cutouts = np.broadcast_to(cutouts, (3, height, width))
        cutouts = np.moveaxis(cutouts, 0, 2)
        img = np.array(img)
        img = img * cutouts
        return Image.fromarray(img.astype('uint8'), 'RGB')

In [3]:
train_transform = transforms.Compose([
            Cutout(num_cutouts=2, size=8, p=0.8),
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
test_transform = transforms.Compose([transforms.ToTensor(),
                                             transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                             ])
train_dataset = datasets.CIFAR10('data/cifar', train=True, download=True, transform=train_transform)

test_dataset = datasets.CIFAR10('data/cifar', train=False, download=True, transform=test_transform)

100%|██████████| 170498071/170498071 [00:02<00:00, 68364667.38it/s]


Extracting data/cifar/cifar-10-python.tar.gz to data/cifar
Files already downloaded and verified


In [4]:
def test(model, batch_size=256):
        model.eval()

        data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in enumerate(data_loader, 0):
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels.flatten()).sum().item()

        model.train()
        return correct / total

In [5]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

In [6]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels,kernel_size=3, stride=1, padding=1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += residual
        out = self.relu(out)

        return out


class ResNet9(nn.Module):
    def __init__(self, activation='relu', fc_size=1024, dropout=0.5):
        super(ResNet9, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=64, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=128, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            BasicBlock(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=256, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=256, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            BasicBlock(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        if activation == "relu":
            self.fc = nn.Sequential(
                nn.Linear(in_features=1024, out_features=fc_size, bias=True),
                nn.Dropout(p=dropout),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(num_features=fc_size, momentum=0.9),
                nn.Linear(in_features=fc_size, out_features=10, bias=True)
            )
        elif activation == "sigmoid":
            self.fc = nn.Sequential(
                nn.Linear(in_features=1024, out_features=fc_size, bias=True),
                nn.Dropout(p=dropout),
                nn.Sigmoid(),
                nn.BatchNorm1d(num_features=fc_size, momentum=0.9),
                nn.Linear(in_features=fc_size, out_features=10, bias=True)
            )
        

    def forward(self, x):
        out = self.conv(x)
        out = out.view(-1, out.shape[1] * out.shape[2] * out.shape[3])
        out = self.fc(out)
        return out

In [7]:
def train(activation="relu", fc_size=1024, dropout=0.25, optimizer="adam", learning_rate=0.001, epochs=75, batch_size=128):
    
    model = ResNet9(activation=activation, fc_size=fc_size, dropout=dropout).to(device)
    print(f"Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
    optimizer = build_optimizer(model, optimizer, learning_rate)
    model.train()
    criterion = nn.CrossEntropyLoss()
    trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    best_accuracy=0
    progress_bar = ProgressBar()

    for epoch in range(1, epochs + 1):
        print('Epoch {}/{}'.format(epoch,epochs))
        running_loss = 0
        epoch_correct = 0
        epoch_total = 0
        for i, (inputs, targets) in enumerate(trainloader):

            inputs, targets = inputs.to(device), targets.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            # Update statistics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            batch_total = targets.size(0)
            batch_correct = predicted.eq(targets).sum().item()


            epoch_total += batch_total
            epoch_correct += batch_correct


            info_str = 'Last batch accuracy: {:.4f} - Running epoch accuracy {:.4f}'.\
                        format(batch_correct / batch_total, epoch_correct / epoch_total)
            progress_bar.update(max_value=len(trainloader), current_value=i, info=info_str)

        progress_bar.new_line()
        test_accuracy = test(model)
        if epoch > 30 and test_accuracy>best_accuracy:
            torch.save(model.state_dict(), "best_model.pth")
            best_accuracy=test_accuracy
            
        print('Test accuracy: {:.2f}% Loss: {:.4f}'.format(test_accuracy*100,running_loss))
    print('Best accuracy: {:.2f}%'.format(best_accuracy*100))
        

In [8]:
train()

Number of trainable parameters: 3499594
Epoch 1/75
 390/391  [==============================] - Last batch accuracy: 0.6375 - Running epoch accuracy 0.4901
Test accuracy: 65.35% Loss: 544.8148
Epoch 2/75
 390/391  [==============================] - Last batch accuracy: 0.6500 - Running epoch accuracy 0.6750
Test accuracy: 71.82% Loss: 360.2338
Epoch 3/75
 390/391  [==============================] - Last batch accuracy: 0.7000 - Running epoch accuracy 0.7341
Test accuracy: 75.71% Loss: 297.3055
Epoch 4/75
 390/391  [==============================] - Last batch accuracy: 0.8000 - Running epoch accuracy 0.7567
Test accuracy: 78.99% Loss: 273.8851
Epoch 5/75
 390/391  [==============================] - Last batch accuracy: 0.6875 - Running epoch accuracy 0.7886
Test accuracy: 81.71% Loss: 238.2189
Epoch 6/75
 390/391  [==============================] - Last batch accuracy: 0.8000 - Running epoch accuracy 0.8115
Test accuracy: 83.83% Loss: 214.9684
Epoch 7/75
 390/391  [====================

In [9]:
def train_best_model(activation="relu", fc_size=1024, dropout=0.25, optimizer="adam", learning_rate=0.001, epochs=75, batch_size=128):
    
    model = ResNet9(activation=activation, fc_size=fc_size, dropout=dropout).to(device)
    model.load_state_dict(torch.load('best_model.pth'))
    print(f"Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
    optimizer = build_optimizer(model, optimizer, learning_rate)
    model.train()
    criterion = nn.CrossEntropyLoss()
    trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    best_accuracy=0
    progress_bar = ProgressBar()

    for epoch in range(1, epochs + 1):
        print('Epoch {}/{}'.format(epoch,epochs))
        running_loss = 0
        epoch_correct = 0
        epoch_total = 0
        for i, (inputs, targets) in enumerate(trainloader):

            inputs, targets = inputs.to(device), targets.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            # Update statistics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            batch_total = targets.size(0)
            batch_correct = predicted.eq(targets).sum().item()


            epoch_total += batch_total
            epoch_correct += batch_correct


            info_str = 'Last batch accuracy: {:.4f} - Running epoch accuracy {:.4f}'.\
                        format(batch_correct / batch_total, epoch_correct / epoch_total)
            progress_bar.update(max_value=len(trainloader), current_value=i, info=info_str)

        progress_bar.new_line()
        test_accuracy = test(model)
        if epoch > 30 and test_accuracy>best_accuracy:
            torch.save(model.state_dict(), "best_model_fine.pth")
            best_accuracy=test_accuracy
            
        print('Test accuracy: {:.2f}% Loss: {:.4f}'.format(test_accuracy*100,running_loss))
    print('Best accuracy: {:.2f}%'.format(best_accuracy*100))
train_best_model(learning_rate=0.0001)        

Number of trainable parameters: 3499594
Epoch 1/75
 390/391  [==============================] - Last batch accuracy: 0.9875 - Running epoch accuracy 0.9847
Test accuracy: 92.95% Loss: 18.0847
Epoch 2/75
 390/391  [==============================] - Last batch accuracy: 0.9750 - Running epoch accuracy 0.9869
Test accuracy: 92.99% Loss: 14.6293
Epoch 3/75
 390/391  [==============================] - Last batch accuracy: 1.0000 - Running epoch accuracy 0.9883
Test accuracy: 93.49% Loss: 13.2499
Epoch 4/75
 390/391  [==============================] - Last batch accuracy: 0.9750 - Running epoch accuracy 0.9883
Test accuracy: 93.39% Loss: 13.4281
Epoch 5/75
 390/391  [==============================] - Last batch accuracy: 0.9875 - Running epoch accuracy 0.9891
Test accuracy: 93.30% Loss: 12.2920
Epoch 6/75
 390/391  [==============================] - Last batch accuracy: 1.0000 - Running epoch accuracy 0.9896
Test accuracy: 93.43% Loss: 11.7368
Epoch 7/75
 390/391  [==========================

In [12]:
model = ResNet9(activation="relu", fc_size=1024, dropout=0.25).to(device)
model.load_state_dict(torch.load('best_model_fine.pth'))
print("Final Model with best Accuracy: {:.2f}%".format(test(model)*100))

Final Model with best Accuracy: 93.88%
